In [5]:
!pip install Crypto
!pip install colorama==0.4.4
!pip install commonmark==0.9.1
!pip install Pillow==8.1.0
!pip install pycryptodome==3.9.9
!pip install pyfiglet==0.8.post1
!pip install Pygments==2.7.4
!pip install rich==9.10.0
!pip install termcolor==1.1.0
!pip install typing-extensions==3.7.4.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pillow: filename=Pillow-8.1.0-cp310-cp310-linux_x86_64.whl size=1163235 sha256=1dd679d24e3667d33e118d4047f079d4eade04514a564e7bd320dcc5a5696188
  Stored in directory: /root/.cache/pip/wheels/69/ff/e2/ff104a7329d86003512672dcce62234921a9bfb347340fd52c
Successfully built Pillow
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.14 requires pillow>=9.0.0, but you have pillow 8.1.0 which is incompatible.
imagei

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 74.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycryptodome: filename=pycryptodome-3.9.9-cp310-cp310-linux_x86_64.whl size=13645248 sha256=5fc90079fe96daba614426a5edb4e3be35bf2f76d5d4b36fe596350e11be3c6a
  Stored in directory: /root/.cache/pip/wheels/49/5b/fb/dca47f0addc9751b38de3a949d0fc968150c3d31ee96faaf6f
Successfully built pycryptodome
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.8/865.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.1/950.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.16.1
    Uninstalling Pygments-2.16.1:
      Successfully uninstalled Pygments-2.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
  Attempting uninstall: rich
    Found existing installation: rich 13.7.1
    Uninstalling rich-13.7.1:
      Successfully uninstalled rich-13.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine 

In [63]:
from PIL import Image
import os.path
from os import path
import math
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto import Random
import base64
from PIL import ImageOps
import imghdr
# from colorama import init
# from termcolor import cprint
# from pyfiglet import figlet_format
from rich import print
from rich.console import Console
from rich.table import Table
from rich.progress import track
# import os
import getpass
import sys
from math import ceil

DEBUG = False
console = Console()
headerText = "M6nMjy5THr2J"
SUPPORTED_FORMATS = ['PNG', 'JPEG', 'JPG', 'TIFF', 'BMP', 'JFIF', 'WEBP']

In [29]:
def encrypt(key, source, encode=True):
    key = SHA256.new(key).digest()  # use SHA-256 over our key to get a proper-sized AES key
    IV = Random.new().read(AES.block_size)  # generate Initialisation Vector (IV)
    encryptor = AES.new(key, AES.MODE_CBC, IV) # AES object
    padding = AES.block_size - len(source) % AES.block_size  # calculate needed padding
    source += bytes([padding]) * padding  # Store the padding size for padding size number of times
    data = IV + encryptor.encrypt(source)  # store the IV at the beginning and encrypt
    return base64.b64encode(data).decode() if encode else data

In [30]:
def decrypt(key, source, decode=True):
    if decode:
        source = base64.b64decode(source.encode())
    key = SHA256.new(key).digest()  # use SHA-256 over our key to get a proper-sized AES key
    IV = source[:AES.block_size]  # extract the IV from the beginning
    decryptor = AES.new(key, AES.MODE_CBC, IV) # AES object
    data = decryptor.decrypt(source[AES.block_size:])  # decrypt
    padding = data[-1]  # pick the padding value from the end
    if data[-padding:] != bytes([padding]) * padding:  # Checking if the padding is valid
        raise ValueError("Invalid padding...")
    return data[:-padding]  # remove the padding and return

In [31]:
# If the image is not in RGB format, convert it to RGB format by creating a new image
# and pasting the current image to it.
def convertToRGB(img):
    try:
        if img.mode == 'RGB':
            return img  # No conversion needed for RGB mode
        rgba_image = img.convert('RGBA')  # Convert to RGBA if not already
        background = Image.new("RGB", rgba_image.size, (255, 255, 255))
        background.paste(rgba_image)
        print("[yellow]Converted image to RGB [/yellow]")
        return background
    except Exception as e:
        print("[red]Couldn't convert image to RGB [/red]- %s" % e)
        return None  # Indicate conversion failure

# Returns the pixel count of the image
def getPixelCount(img):
    width, height = Image.open(img).size
    return width*height

In [32]:
def calculate_required_dimensions(message_length, max_pixels_per_character):
    # Calculate the minimum required dimensions for the cover image
    pixels_needed = message_length * max_pixels_per_character
    width = int((pixels_needed ** 0.5) + 0.5)  # Round up to the nearest integer
    height = (pixels_needed + width - 1) // width  # Ensure all pixels are used without distortion
    return width, height

In [33]:
def compressImage(image_path):
    # Compress the image using lossless compression techniques
    try:
      try:
        img = Image.open(image_path)
      except Exception as e:
        print(f"Error opening the image: {e}")
          # Check if the image format is supported
      if img.format.upper() not in SUPPORTED_FORMATS:
          raise ValueError(f"Unsupported image format: {image.format}")

      filename, extension = os.path.splitext(image_path)  # Split filename and extension
      compressed_path = f"{filename}-compress.png"  # Generate compressed image path

      if img.format == "PNG":
        print(f"[yellow]Image is already in PNG format. Using it as is. [/yellow]")
        return compressed_path  # No need to save again, return path
      else:
        img.save(compressed_path, "PNG")  # Save as PNG for compression (if not already)
        print(f"[yellow]Image compressed and saved to {compressed_path}[/yellow]")
        return compressed_path
    except Exception as e:
      print(f"[red]Error compressing image: {e}[/red]")
      return None  # Indicate failure

In [34]:
def encodeImage(image,message,filename):
    with console.status("[green]Encoding image..") as status:
        try:
            width, height = image.size
            pix = image.getdata()

            current_pixel = 0
            tmp=0
            # three_pixels = []
            x=0
            y=0
            for ch in message:
                binary_value = format(ord(ch), '08b')

                # For each character, get 3 pixels at a time
                p1 = pix[current_pixel]
                p2 = pix[current_pixel+1]
                p3 = pix[current_pixel+2]

                three_pixels = [val for val in p1+p2+p3]

                for i in range(0,8):
                    current_bit = binary_value[i]

                    # 0 - Even
                    # 1 - Odd
                    if current_bit == '0':
                        if three_pixels[i]%2!=0:
                            three_pixels[i]= three_pixels[i]-1 if three_pixels[i]==255 else three_pixels[i]+1
                    elif current_bit == '1':
                        if three_pixels[i]%2==0:
                            three_pixels[i]= three_pixels[i]-1 if three_pixels[i]==255 else three_pixels[i]+1

                current_pixel+=3
                tmp+=1

                #Set 9th value
                if(tmp==len(message)):
                    # Make as 1 (odd) - stop reading
                    if three_pixels[-1]%2==0:
                        three_pixels[-1]= three_pixels[-1]-1 if three_pixels[-1]==255 else three_pixels[-1]+1
                else:
                    # Make as 0 (even) - continue reading
                    if three_pixels[-1]%2!=0:
                        three_pixels[-1]= three_pixels[-1]-1 if three_pixels[-1]==255 else three_pixels[-1]+1


                if DEBUG:
                    print("Character: ",ch)
                    print("Binary: ",binary_value)
                    print("Three pixels before mod: ",three_pixels)
                    print("Three pixels after mod: ",three_pixels)


                three_pixels = tuple(three_pixels)

                st=0
                end=3

                for i in range(0,3):
                    if DEBUG:
                        print("Putting pixel at ",(x,y)," to ",three_pixels[st:end])

                    image.putpixel((x,y), three_pixels[st:end])
                    st+=3
                    end+=3

                    if (x == width - 1):
                        x = 0
                        y += 1
                    else:
                        x += 1

            encoded_filename = filename.split('.')[0] + "-enc.png"
            image.save(encoded_filename)
            print("\n")
            print("[yellow]Original File: [u]%s[/u][/yellow]"%filename)
            print("[green]Image encoded and saved as [u][bold]%s[/green][/u][/bold]"%encoded_filename)

        except Exception as e:
            print("[red]An error occured - [/red]%s"%e)
            sys.exit(0)

In [58]:
def decodeImage(image):
    with console.status("[green]Decoding image..") as status:
        try:
            pix = image.getdata()
            current_pixel = 0
            decoded=""
            data_found = False  # Flag to indicate if any data is found in the image
            while True:
                try:
                  # Get 3 pixels each time
                  binary_value=""
                  p1 = pix[current_pixel]
                  p2 = pix[current_pixel+1]
                  p3 = pix[current_pixel+2]
                  three_pixels = [val for val in p1+p2+p3]

                  for i in range(0,8):
                      if three_pixels[i]%2==0:
                          # add 0
                          binary_value+="0"
                      elif three_pixels[i]%2!=0:
                          # add 1
                          binary_value+="1"


                  #Convert binary value to ascii and add to string
                  binary_value.strip()
                  ascii_value = int(binary_value,2)
                  decoded+=chr(ascii_value)
                  current_pixel+=3

                  if DEBUG:
                      print("Binary: ",binary_value)
                      print("Ascii: ",ascii_value)
                      print("Character: ",chr(ascii_value))

                  if three_pixels[-1]%2!=0:
                      # stop reading
                      data_found =True
                      break
                except(ValueError, TypeError) as e:  # Catch potential errors within the loop
                    # print("[red]Error processing image data. The provided image might not be a valid stego image.[/red]")
                    raise Exception(f"[red]Error processing image data. The provided image might not be a valid stego image.[/red]")
                    break

            if not data_found:
                raise ValueError("No data found in the provided image.")

            # print("Decoded: %s"%decoded)
            return decoded
        except Exception as e:
            print("[red]An error occured - [/red]%s"%e)
            sys.exit()

In [71]:
def main():
    # insertHeaders(img)

    while True:
        print("[cyan]Choose one: [/cyan]")
        op = int(input("1. Encode\n2. Decode\n3. Exit\n>>"))

        if op == 1:

            print("[cyan]Message to be hidden: [/cyan]")
            message = input(">>")
            if not message:
                raise ValueError("Message cannot be empty.")

            message = headerText + message

            # Calculate required dimensions for cover image
            max_pixels_per_character = 3 * 8  # Assuming 3 pixels per color channel and 8 bits per channel
            # aspect_ratio = 1.0  # Assuming square pixels, adjust if necessary
            required_width, required_height = calculate_required_dimensions(len(message), max_pixels_per_character)
            print(f"[yellow]Minimum required dimensions for cover image: [bold]{required_width} x {required_height} pixels[/bold][/yellow]")

            print("[cyan]Image path (with extension): [/cyan]")
            img = input(">>")

            # Validating Image Format
            if not (path.exists(img)):
                raise Exception("Image not found!")

            image_format = imghdr.what(img)
            if image_format not in {'png', 'jpg', 'jpeg', 'tiff', 'bmp', 'jfif', 'webp'}:
                raise ValueError("Invalid image format. Supported formats are: png, jpg, jpeg, tiff, bmp, jfif, webp.")
            # Check if message fits in the image
            img_width, img_height = Image.open(img).size
            compressed_image_path = compressImage(img)
            # if compressed_image_path:
            #   print(f"Compressed image saved at: {compressed_image_path}")
            #   compressed_image=Image.open(compressed_image_path)
            # else:
            #   raise Exception("Error compressing image.")

            # if not (path.exists(compressed_image_path)):
            #   raise Exception("Error compressing image.")
            # else:
            #   print(f"Compressed image saved at: {compressed_image_path}")
            #   compressed_image=Image.open(compressed_image_path)
            # compressed_width, compressed_height = compressed_image.size

            if img_width < required_width or img_height < required_height:
               raise Exception(f"The provided image dimensions ({img_width} x {img_height} pixels) are smaller than the suggested dimensions ({required_width} x {required_height} pixels). Please provide an image with suitable dimensions.")

            # if compressed_width < required_width or compressed_height < required_height:
            #     raise Exception(f"The provided image dimensions ({img_width} x {img_height} pixels) are smaller than the suggested dimensions ({required_width} x {required_height} pixels). Please provide an image with suitable dimensions.")

            max_message_length = (img_width * img_height * 3) // 8 - len(headerText)  # Maximum message length in characters
            if len(message) > max_message_length:
                raise Exception(f"Given message is too long to be encoded in the image.\nTry resizing the image to accommodate a larger message.\nCurrent maximum message length: {max_message_length} characters.")

            # max_message_length = (compressed_width * compressed_height * 3) // 8 - len(headerText)
            # if len(message) > max_message_length:
            #     raise Exception(f"The provided image dimensions after compression ({compressed_width} x {compressed_height} pixels) are smaller than the suggested dimensions for the message. Please provide an image with suitable dimensions.")

            password = None
            while 1:
                print("[cyan]Password to encrypt: [/cyan]")
                password = getpass.getpass(">>")
                # if password == "":
                #     break
                if password:
                  print("[cyan]Re-enter Password: [/cyan]")
                  confirm_password = getpass.getpass(">>")
                  if password != confirm_password:
                      print("[red]Passwords don't match try again [/red]")
                  else:
                      break
                else:
                  raise ValueError("Password cannot be empty.")

            cipher = ""
            if password != "":
                cipher = encrypt(key=password.encode(), source=message.encode())
                # Add header to cipher
                cipher = headerText + cipher
            else:
                cipher = message

            if DEBUG:
                print("[yellow]Encrypted : [/yellow]", cipher)

            try:
                image = Image.open(img)
            except Exception as e:
                print(f"Error opening the image: {e}")

            image.filename=img
            print("[yellow]Image Mode: [/yellow]%s"%image.mode)
            if image.mode!='RGB':
                image = convertToRGB(image)
            newimg = image.copy()
            encodeImage(image=newimg,message=cipher,filename=img)

        elif op == 2:
            print("[red]Stego Image must be in png format[/red]")
            print("[cyan]Image path (with extension): [/cyan]")
            img = input(">>")
            if not (path.exists(img)):
                raise Exception("Image not found!")

            password=None
            while True:
              print("[cyan]Enter password : [/cyan]")
              password_input = getpass.getpass(">>")
              if password_input:
                password = password_input
                break
              else:
                raise ValueError("Password cannot be empty.")

            try:
                image = Image.open(img)
            except Exception as e:
                print(f"Error opening the image: {e}")

            # Check if the image format is PNG
            if image.format.upper() != 'PNG':
                raise ValueError("The stego image must be in PNG format.")

            # Check if the image contains the expected header
            # header = image.info.get('header')
            # if header != headerText:
            #     raise ValueError("The provided image is not a stego image.")

            cipher = decodeImage(image)

            header = cipher[:len(headerText)]

            if header.strip() != headerText:
                print("[red]Invalid data![/red]")
                sys.exit(0)

            print()

            if DEBUG:
                print("[yellow]Decoded text: %s[/yellow]" % cipher)

            decrypted = ""

            if password != "":
                cipher = cipher[len(headerText):]
                print("cipher : ", cipher)
                try:
                    decrypted = decrypt(key=password.encode(), source=cipher)
                except Exception as e:
                    print("[red]Wrong password![/red]")
                    sys.exit(0)

            else:
                decrypted = cipher

            header = decrypted.decode()[:len(headerText)]

            if header != headerText:
                print("[red]Wrong password![/red]")
                sys.exit(0)

            decrypted = decrypted[len(headerText):].decode("utf-8")

            print("[green]Decoded Text: \n[bold]%s[/bold][/green]" % decrypted)

        elif op == 3:
            break

        else:
            print("Incorrect Choice")
        print("\n")


if __name__ == "__main__":
    os.system('cls' if os.name == 'nt' else 'clear')
    # cprint(figlet_format('IMGHIDE!', font='starwars'),'yellow', attrs=['bold'])
    # print_credits()
    print()
    print()

    main()



Choose one: 
1. Encode
2. Decode
3. Exit
>>1
Message to be hidden: 
>>/content/smiley.jpeg
Minimum required dimensions for cover image: 28 x 28 pixels
Image path (with extension): 
>>/content/smiley.jpeg
Image compressed and saved to /content/smiley-compress.png
Password to encrypt: 
>>··········
Re-enter Password: 
>>··········
Image Mode: RGB


Original File: /content/smiley.jpeg
Image encoded and saved as /content/smiley-enc.png


Choose one: 
1. Encode
2. Decode
3. Exit
>>2
Stego Image must be in png format
Image path (with extension): 
>>/content/smiley-enc.png
Enter password : 
>>··········

cipher :  lAepw1q7aF+mN88yrvXfxeJb5nA2gj9nvgoXot240B5k63PAaujiM8nGfI9n3Xzv7Utw35
P9Ve1cX6uDoaDHhQ==
Decoded Text: 
/content/smiley.jpeg


Choose one: 
1. Encode
2. Decode
3. Exit
>>3
